In [1]:
# EXTRACCION

In [2]:
!pip install spotipy python-dotenv
import base64
from requests import post,get
import json
import numpy as np
import pandas as pd

In [3]:
# Obtener las credenciales del archivo .env
CLIENT_ID='540ba0d5c2564cdeb246ace0d1ada7cf'
CLIENT_SECRET='dcad0c99f6304377b3980b8a811ffe9e'

# Verificar que las variables se cargaron correctamente
if not CLIENT_ID or not CLIENT_SECRET:
    raise ValueError("CLIENT_ID y CLIENT_SECRET deben estar configurados en el archivo .env")

def get_token():
    auth_string = CLIENT_ID + ":" + CLIENT_SECRET
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result
    return token["access_token"]

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def search_for_playlist(token, playlist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={playlist_name}&type=playlist&limit=1"
    query_url = url + query
    result = get(query_url, headers=headers)
    json_result = result.json()
    return json_result

def get_playlist(token, playlist_id):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = result.json()
    return json_result

def get_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = result.json()
    return json_result

def get_features(token, songs_id):
    url = f"https://api.spotify.com/v1/audio-features?ids={songs_id}"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = result.json()
    return json_result

In [4]:
lista_paises={'Los 40 Principales':"6hN982OqftxyShJ28FcAIc",'UK Top 100':"2JyhlK34xSSXR4OQ3WflEq",'Billboard Hot 100':"6UeSakyzhiEt4NB3UAd6NQ",'Shazam Top 200':'1YvHZR6lwysBB6UNUa800r'}

In [5]:
token = get_token()
print("¡Token obtenido exitosamente!")
print(f"Token (primeros 20 caracteres): {token[:20]}...")

¡Token obtenido exitosamente!
Token (primeros 20 caracteres): BQCEn0Ru-ESHOOYgGlCu...


In [6]:
columns = ["top", "track_name", "track_popularity", "duration_ms", "artist_name", 
           "artist_genres", "artist_popularity", "feats", "explicit", "album", 
           "type", "relase_date", "track_id", "artist_id", "list"]
df_playlist = pd.DataFrame(columns=columns)

In [7]:
jsons = []
for playlist_name, playlist_id in lista_paises.items():
    playlist_json = get_playlist(token, playlist_id)
    playlist_json['playlist name']=playlist_name
    jsons.append(playlist_json)

In [8]:
for json in jsons:
    current_playlist_name=json['playlist name']
    track_info = json["tracks"]["items"]
    playlist_data = []
    for i in range(len(json["tracks"]["items"])):
        feats = []
        data = {
            "top": i+1,
            "track_name": track_info[i]["track"]["name"],
            "track_popularity": track_info[i]["track"]["popularity"],
            "artist_name": track_info[i]["track"]["artists"][0]["name"],
            "artist_genres": "",
            "duration_ms": track_info[i]["track"]["duration_ms"],
            "artist_popularity": "",
            "feats": "",
            "explicit": track_info[i]["track"]["explicit"],
            "album": track_info[i]["track"]["album"]["name"],
            "type": track_info[i]["track"]["album"]["album_type"],
            "relase_date": track_info[i]["track"]["album"]["release_date"],
            "track_id": track_info[i]["track"]["id"],
            "artist_id": track_info[i]["track"]["artists"][0]["id"],
            "list": current_playlist_name
            }

        artist_info = get_artist(token, data["artist_id"])
        data["artist_popularity"] = artist_info["popularity"]

        if len(artist_info["genres"]) == 1:
            data["artist_genres"] = artist_info["genres"][0]
        elif len(artist_info["genres"]) > 1:
            generes = artist_info["genres"]
            data["artist_genres"] = "|".join(generes)
        else:
            data["artist_genres"] = np.nan

        if len(track_info[i]["track"]["artists"]) > 1:
            for j in range(len(track_info[i]["track"]["artists"])-1):
                feats.append(track_info[i]["track"]["artists"][j+1]["name"])
                data["feats"] = "|".join(feats)
        else:
            data["feats"] = np.nan
        feats = []

        playlist_data.append(data)

        
    # CONCAT
    df_temp = pd.DataFrame(playlist_data)
        
    df_playlist = pd.concat([df_playlist, df_temp], ignore_index=True)

In [9]:
df_playlist.to_csv("top_charts.csv", index=False, encoding='utf-8')
print("CSV exportado")

CSV exportado
